# Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

If any error, you might need to specify the default artifact root (usually `mlruns`):
`mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local`.

In [1]:
# Necessary import
import mlflow
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from mlflow.tracking import MlflowClient

# Set the tracking uri as local server
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
# Get the tracking uri
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
# List experiments
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1748179921522, experiment_id='1', last_update_time=1748179921522, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1748179296216, experiment_id='0', last_update_time=1748179296216, lifecycle_stage='active', name='Default', tags={}>]

## Creating an experiment and logging a new run

In [4]:
# Set a ML experiment
mlflow.set_experiment("my-experiment-1")

# Start an experiment run 
with mlflow.start_run():
    # Load the iris data set features and target
    X, y = load_iris(return_X_y = True)

    # Model parameters
    params = {"C": 0.1, "random_state": 42}
    # Log the model parameters
    mlflow.log_params(params)

    # Initialize and train the logistic model
    lr = LogisticRegression(**params).fit(X, y)
    # Make predictions
    y_pred = lr.predict(X)
    # Log the model accuracy
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    # Log the model
    mlflow.sklearn.log_model(lr, artifact_path = "models")
    # Check the mlfow artiact uri
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")


2025/05/25 14:40:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/1/e7fde540966a4e6b84aca9e80b2c770a/artifacts'
🏃 View run mysterious-gnat-17 at: http://127.0.0.1:5000/#/experiments/1/runs/e7fde540966a4e6b84aca9e80b2c770a
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [5]:
# Show the updated lists of experiments
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1748179921522, experiment_id='1', last_update_time=1748179921522, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1748179296216, experiment_id='0', last_update_time=1748179296216, lifecycle_stage='active', name='Default', tags={}>]

## Interacting with the model registry

In [6]:
# Create an mlflow client
client = MlflowClient("http://127.0.0.1:5000")

In [7]:
# List registered models
client.search_registered_models()

[]

In [8]:
# Get the information about the first experiment run
run_id = client.search_runs(experiment_ids = '1')[0].info.run_id
# Register a model
mlflow.register_model(
    model_uri = f"runs:/{run_id}/models",
    name = 'iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/05/25 14:40:53 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1748180453191, current_stage='None', description='', last_updated_timestamp=1748180453191, name='iris-classifier', run_id='e7fde540966a4e6b84aca9e80b2c770a', run_link='', source='mlflow-artifacts:/1/e7fde540966a4e6b84aca9e80b2c770a/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [9]:
# Updated list of registered models
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1748180453180, description='', last_updated_timestamp=1748180453191, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1748180453191, current_stage='None', description='', last_updated_timestamp=1748180453191, name='iris-classifier', run_id='e7fde540966a4e6b84aca9e80b2c770a', run_link='', source='mlflow-artifacts:/1/e7fde540966a4e6b84aca9e80b2c770a/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>], name='iris-classifier', tags={}>]

---